In [ ]:
!unzip /content/drive/MyDrive/vlg_pro/Generative_AI_Dataset.zip

Archive:  /content/drive/MyDrive/vlg_pro/Generative_AI_Dataset.zip
  inflating: Dataset/solution_format.csv  
  inflating: Dataset/test.csv        
  inflating: Dataset/train.csv       


In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('/content/Dataset/train.csv')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np


class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.labels = self.data['labels']
        self.features = self.data.drop('labels', axis=1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.labels[idx]
        features = self.features.iloc[idx].values.reshape((20, 20,3))
        features = features.transpose((2,0,1))
        sample = {'features': torch.Tensor(features), 'label': label}
        return sample


def custom_collate_fn(batch):
    features = [item['features'] for item in batch]
    labels = [item['label'] for item in batch]
    features = torch.stack(features)
    labels = torch.tensor(labels)
    return {'features': features, 'labels': labels}


dataset = CustomDataset(train_df)
dataloader = DataLoader(dataset, batch_size=1, collate_fn=custom_collate_fn)

In [ ]:
import torch.nn as nn
import torchvision.models as models
model = models.vgg16(pretrained=True)
feature_extractor = nn.Sequential(*list(model.features.children())[:-1])



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
 

In [ ]:
model2 = models.resnet18(pretrained=True)
feature_extractor_2 = nn.Sequential(*list(model2.children())[:-1])

In [ ]:
ext_f = []
for batch in dataloader :
  images = batch['features']
  feature_extractor_2.eval()
  feature_extractor.eval()
  d=feature_extractor_2(images).detach().cpu().numpy().reshape(512)
  #d2=feature_extractor(images).detach().cpu().numpy().reshape(512)
  feat =d#np.concatenate((d, d2), axis=0)
  ext_f.append(feat)

In [ ]:
ext_df = pd.DataFrame(ext_f)
ext_df.columns = ext_df.columns.astype(str)
ext_df = pd.concat([ext_df,train_df.drop('labels',axis=1)],axis=1)
ext_df.head()

,0,1,2,3,4,5,6,7,8,9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,0.942797,1.349738,0.043392,0.927741,0.026253,0.0,0.313777,0.000000,0.000000,0.0,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,0.461580,0.779857,0.351044,1.053287,0.195577,0.0,0.443317,0.065541,0.138443,0.0,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,0.713785,0.724120,0.319372,1.355888,0.000000,0.0,0.339933,0.000000,0.281310,0.0,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,0.784277,0.638512,0.000000,1.099986,0.072516,0.0,0.343079,0.000000,0.011497,0.0,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0.994810,0.930024,0.327882,0.277144,0.322679,0.0,0.580977,0.166276,0.062565,0.0,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
knn = KNeighborsClassifier(n_neighbors= 7, p= 1, weights= 'uniform')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ext_df, train_df['labels'], test_size=0.2, random_state=42)


In [ ]:
knn.fit(ext_df,train_df['labels'])
knn.score(X_test,y_test)

0.9133333333333333

In [ ]:
pred = knn.predict(X_test)
f1_score(y_test,pred)

0.8253358925143953

In [ ]:
test_df = pd.read_csv('/content/Dataset/test.csv')
test=test_df.drop('id',axis=1)
class TestCustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.features = self.data

    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        features = self.features.iloc[idx].values.reshape((3,20, 20))
        sample = {'features': torch.Tensor(features)}
        return sample


def custom_collate_fn(batch):
    features = [item['features'] for item in batch]
    features = torch.stack(features)
    return {'features': features}
test_dataset = TestCustomDataset(test)
test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=custom_collate_fn)

In [ ]:
t_ext_f = []
for batch in test_dataloader:
  images = batch['features']
  feature_extractor_2.eval()
  feature_extractor.eval()
  d=feature_extractor_2(images).detach().cpu().numpy().reshape(512)
  #d2=feature_extractor(images).detach().cpu().numpy().reshape(512)
  feat =d#np.concatenate((d, d2), axis=0)
  t_ext_f.append(feat)

In [ ]:
t_ext_df = pd.DataFrame(t_ext_f)
t_ext_df.columns = t_ext_df.columns.astype(str)
t_ext_df = pd.concat([t_ext_df,test_df.drop('id',axis=1)],axis=1)
t_ext_df.head()

,0,1,2,3,4,5,6,7,8,9,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,1.000861,2.009952,0.317756,0.866879,0.358176,0.0,0.819393,0.000000,0.011622,0.0,...,-0.776403,-0.662884,-0.257091,-1.168413,0.223260,-0.482520,-0.085453,-0.382265,-0.539349,-1.682404
1,0.757142,0.942584,0.117522,1.147421,0.126367,0.0,0.363663,0.000000,0.000000,0.0,...,0.379635,-1.760084,1.125450,-0.328047,-0.880305,-1.257607,0.964312,2.021104,0.655021,-0.423029
2,0.963658,0.884242,0.000000,0.429560,0.415525,0.0,0.494465,0.000000,0.128982,0.0,...,1.165254,-1.373589,-0.483701,-0.964782,-0.869555,0.066040,-0.444567,-0.531935,-0.878660,1.099488
3,0.887752,0.244355,0.000000,0.585474,0.253120,0.0,0.449613,0.089257,0.087119,0.0,...,-0.442288,-2.794472,-0.763468,-0.789832,-0.113209,-2.703150,-2.058728,1.070627,-0.458045,-0.435825
4,0.757546,0.991289,0.195527,1.102310,0.102083,0.0,0.520727,0.232494,0.064232,0.0,...,-2.624450,-3.200223,0.711422,-0.190394,0.337224,-1.656639,0.707360,-0.562290,1.471181,-0.192000


In [ ]:
pred = knn.predict(t_ext_df)
pred.mean()

0.24977777777777777

In [ ]:
sub=pd.DataFrame([])
sub['id']=test_df['id']
sub['labels']=pred

In [ ]:
sub.to_csv('knn_sub9.csv',index=False)
